In [1]:
import matplotlib.pyplot as plt
import numpy as np
from astropy.io import fits
import os
from astropy.table import Table, hstack, vstack
from matplotlib.colors import LogNorm
from scipy.stats import kde
import healpy as hp
from numpy import empty
import astropy.coordinates
import pandas as pd
import glob 
from astropy.io.fits import getheader
from astropy.table import Table
from astropy import units as u
from astropy.coordinates import SkyCoord
import fitsio
import time as time
from joblib import Parallel, delayed
from tqdm import tqdm

In [2]:
###load the catalog you want to obtain more information
data = Table.read('mytable.fits', format='fits')
df_my = data.to_pandas()
df_my.info()

###select just the collumns needed for this file
objid = np.array(data['NUMBER'])
ra_my = np.array(data['ALPHA_J2000'])
dec_my = np.array(data['DELTA_J2000'])
#disk = np.array(data['P_disk'])



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 964 entries, 0 to 963
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   NUMBER         964 non-null    int32  
 1   ALPHA_J2000    964 non-null    float64
 2   DELTA_J2000    964 non-null    float64
 3   ISOAREA_IMAGE  964 non-null    int32  
dtypes: float64(2), int32(2)
memory usage: 22.7 KB


In [3]:
#####loading the fullcats file
filenames = glob.glob("fullcats/*.fits")
print(filenames)
path_output = 'out_crossmatch/'

['fullcats/D00817272_r_09_r1p1_fullcat.fits', 'fullcats/D00817688_r_29_r1p1_fullcat.fits', 'fullcats/D00850390_r_42_r1p1_fullcat.fits', 'fullcats/D00817258_r_59_r1p1_fullcat.fits', 'fullcats/D00817257_r_13_r1p1_fullcat.fits', 'fullcats/D00850390_r_42_r1p1_fullcat (copy).fits', 'fullcats/D00850390_r_42_r1p1_fullcat (another copy).fits']


In [4]:
####its a funtions to obtain just the filename instead of the full path
def getName(string):
    name = string.split('/')[1] ###how many bars has in filenames
    return name

In [5]:
def Crossmatch(filenames,max_separation,step):
    f = filenames[step]
    #print(f)
    data_fullcats = fitsio.read(f,columns=['ALPHA_J2000','DELTA_J2000','ISOAREA_IMAGE','ISOAREAF_IMAGE','KRON_RADIUS','FLAGS'], ext=2)
    #df_fullcats = pd.DataFrame(data_fullcats.byteswap().newbyteorder())
    ###define the collumns you want to match
    #inicio_individual = time.time()
    ra_fullcats = np.array(data_fullcats['ALPHA_J2000'])
    dec_fullcats = np.array(data_fullcats['DELTA_J2000'])
    ISOAREA_IMAGE = np.array(data_fullcats['ISOAREA_IMAGE'])
    ISOAREAF_IMAGE = np.array(data_fullcats['ISOAREAF_IMAGE'])
    KRON_RADIUS = np.array(data_fullcats['KRON_RADIUS'])
    FLAGS_fullcat = np.array(data_fullcats['FLAGS'])
    
    #####running the match
    c_my = SkyCoord(ra=ra_my*u.degree, dec=dec_my*u.degree)
    c_fullcats = SkyCoord(ra=ra_fullcats*u.degree, dec=dec_fullcats*u.degree)
    idx, d2d, d3d = c_my.match_to_catalog_sky(c_fullcats)
    print(d2d.value)
    sep_constraint = d2d < max_separation
    #final_individual = time.time()
    #print("time to peform the individual match for", getName(f),"=",final_individual - inicio_individual, "seconds")
    if len(idx[sep_constraint]) > 0:
        idx = idx[sep_constraint]
        objid_match = objid[sep_constraint]
        objid_match_f = objid_match.reshape(-1,1)
        ra_my_match = ra_my[sep_constraint]
        ra_my_match_f = ra_my_match.reshape(-1,1)
        dec_my_match = dec_my[sep_constraint]
        dec_my_match_f = dec_my_match.reshape(-1,1)
        
        ra_fullcats_match = ra_fullcats[idx[sep_constraint]]
        ra_fullcats_match_f = ra_fullcats_match.reshape(-1,1)
        dec_fullcats_match = dec_fullcats[idx[sep_constraint]]
        dec_fullcats_match_f = dec_fullcats_match.reshape(-1,1)
        ISOAREA_IMAGE_match = ISOAREA_IMAGE[idx[sep_constraint]]
        ISOAREA_IMAGE_match_f = ISOAREA_IMAGE_match.reshape(-1,1)
        ISOAREAF_IMAGE_match = ISOAREAF_IMAGE[idx[sep_constraint]]
        ISOAREAF_IMAGE_match_f = ISOAREAF_IMAGE_match.reshape(-1,1)
        KRON_RADIUS_match = KRON_RADIUS[idx[sep_constraint]]
        KRON_RADIUS_match_f = KRON_RADIUS_match.reshape(-1,1)
        FLAGS_fullcat_match = FLAGS_fullcat[idx[sep_constraint]]
        FLAGS_fullcat_match_f = FLAGS_fullcat_match.reshape(-1,1)

        ####saving all the columns into a single array
        con = np.concatenate((objid_match_f,ra_my_match_f,dec_my_match_f,ra_fullcats_match_f,dec_fullcats_match_f,
                              ISOAREA_IMAGE_match_f,ISOAREAF_IMAGE_match_f,KRON_RADIUS_match_f,FLAGS_fullcat_match_f), axis=1)
        ###saving the con array  into a table
        df_out = pd.DataFrame(con, columns= ['objID','ra','dec','ra_fullcats','dec_fullcats','ISOAREA','ISOAREAF','Kron_radius','flag'] )
        
        t = Table.from_pandas(df_out)
        t.write(path_output+getName(f), overwrite=True)
    else:
        print("Not enough objects inside the crossmatch Radius")
        
    


In [6]:
max_separation = 2.0*u.arcsec
num_process = 2 # seta numero dos cors aqui
inicio_match = time.time()

Parallel(n_jobs=num_process)(delayed(Crossmatch)(filenames, max_separation, step) for step in tqdm(range(len(filenames))))
final_match = time.time()


100%|██████████| 7/7 [00:01<00:00,  6.77it/s]
